In [1]:
import cv2
import os

# this func is created to detect number_plates from image or video
def number_plate_detector(source_path,results_path,show = False):
    
    # using opencv builtin classifier for number plate detection
    n_plate_Cascade = cv2.CascadeClassifier("haarcascades/haarcascade_russian_plate_number.xml")
    video = False
    
    # for image
    if source_path.lower().endswith(('.jpg', '.png')):
        img = cv2.imread(source_path)
        imgGray =  cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        numberPlates = n_plate_Cascade.detectMultiScale(imgGray,1.1,4)
        count = 1
        for (x,y,w,h) in numberPlates:
            area = w*h
            if area>500:
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
                cv2.putText(img,'Number Plate',(x,y-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,[255,0,0],2)
                nplateimage = img[y:y+h,x:x+w] # cropping number plate
                platepath = results_path.split('.')[0]+f'_cropped_{count}.png'
                cv2.imwrite(platepath,nplateimage) # saving cropped number plate
                count += 1  
        
        if show :
            cv2.imshow("Result",img)
            cv2.waitKey(0) 
        cv2.imwrite(results_path,img) # saving image
        
    # for video
    elif source_path.lower().endswith(('.mp4')):
        video = True
        cap = cv2.VideoCapture(source_path)
        frameWidth = 640
        frameHeight = 480 
        cap.set(3,frameWidth) # width has id no. 3
        cap.set(4,frameHeight) # height has id no. 4
        cap.set(10,130) # brightness has id no. 10
        ret, frame = cap.read() 
        output_video = cv2.VideoWriter(results_path,
                              cv2.VideoWriter_fourcc(*'MP4V'),
                              25,
                              (frame.shape[1],frame.shape[0]))
        ret = True
        while ret:
            ret,frame = cap.read()
            if frame is not None: 
                imgGray =  cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
                numberPlates = n_plate_Cascade.detectMultiScale(imgGray,1.1,4)
                for (x,y,w,h) in numberPlates:
                    area = w*h
                    if area>500:
                        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
                        cv2.putText(frame,'Number Plate',(x,y-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,[255,0,0],2)
                        nplateimage = frame[y:y+h,x:x+w]
                if show:
                    cv2.imshow("Result",frame)
                    if cv2.waitKey(1) & 0xFF == ord('n'):
                        break
                output_video.write(frame) # saving video
                
                
    else:
            print('irrelevantfile')
    
    if video == True:
        cap.release()
        output_video.release()
    cv2.destroyAllWindows()

In [2]:
# for test_img

In [3]:
source_path = os.path.join('test_img','test_img.png')
results_path = os.path.join('test_img','result.png')

In [4]:
number_plate_detector(source_path,results_path)

In [5]:
# for test_video

In [6]:
source_path = os.path.join('test_video','video.mp4')
results_path = os.path.join('test_video','result.mp4')

In [7]:
number_plate_detector(source_path,results_path)